In [1]:
import sys
sys.path.append('../../build/mechsystem')

from mass_spring import *

from pythreejs import *

import math

## Spinning Top
This file is based on the mass_spring.ipynb file. It was modified to model a spinning top.

In [ ]:
mss = MassSpringSystem3d()
mss.gravity = (0,-9.81,0) #gravity in comparison to mass_spring in other direction!
#mss.gravity = (0,-9.81,0)

height = 4.0
stiffGlob = 20000
tangentSpeed = 2000

f1 = mss.add (Fix( (0,0,0)))
m1 = mss.add (Mass(0.5, (0.0,0.0,height)))
m2 = mss.add (Mass(1, (1.0,0.0,height)))
m3 = mss.add (Mass(1, (-0.5,-0.866025,height)))
m4 = mss.add (Mass(1, (-0.5, 0.866025,height)))

mss.add (Spring(1, stiffGlob, (f1, m1)))
#mss.add(Rigid(1, f1, m1))
mss.add (Spring(1, stiffGlob, (m1, m2)))
mss.add (Spring(1, stiffGlob, (m1, m3)))
mss.add (Spring(1, stiffGlob, (m1, m4)))
#mss.add (Spring(1, stiffGlob, (f1, m2)))
#mss.add (Spring(1, stiffGlob, (f1, m3)))
#mss.add (Spring(1, stiffGlob, (f1, m4)))
#mss.add (Spring(1, stiffGlob, (m2, m3)))
#mss.add (Spring(1, stiffGlob, (m3, m4)))
#mss.add (Spring(1, stiffGlob, (m4, m2)))



3

In [3]:
masses = []
for m in mss.masses:
    masses.append(Mesh(SphereBufferGeometry(0.2, 16, 16), MeshStandardMaterial(color='red'), position=m.pos)) 

mss.masses[-3].vel = (0.0, -tangentSpeed, 0.0) # setting an initial velocity to the mass element m2 (third last)
mss.masses[-2].vel = (-tangentSpeed * math.cos(60*math.pi/180), tangentSpeed * math.sin(60*math.pi/180), 0.0)
mss.masses[-1].vel = (tangentSpeed * math.sin(60*math.pi/180), tangentSpeed * math.cos(60*math.pi/180), 0.0)

#mass_object = mss.masses[-3]
#mass_object.vel = (0.0, 0.0, 0.0) 

fixes = []
for f in mss.fixes:
    fixes.append(Mesh(SphereBufferGeometry(0.2, 32, 16), MeshStandardMaterial(color='blue'), position=f.pos)) 

springpos = []
for s in mss.springs:
    pA = mss[s.connectors[0]].pos
    pB = mss[s.connectors[1]].pos
    springpos.append ([ pA, pB ] ) 

springgeom = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan') # only visible in longr chain elements
springs = LineSegments2(springgeom, m2)    

axes = AxesHelper(1)

In [4]:
view_width = 600
view_height = 400

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height, up=[0,0,1])
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [5]:
from time import sleep
for i in range(1000):
    mss.simulate (0.0001, 500)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        springpos.append ([ pA, pB ]) 
    springs.geometry = LineSegmentsGeometry(positions=springpos)
    sleep(0.01)

KeyboardInterrupt: 